# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
#NOTE: Identation is not very important here because SQL does not rely on proper Identation
context = [ {'role':'system', 'content':"""
-- Table: employees
CREATE TABLE employees (
    ID_usr INT,
    name VARCHAR(100),
    age INT
);
-- Sample Rows:
INSERT INTO employees (ID_usr, name, age) VALUES (1, 'Alice', 30);
INSERT INTO employees (ID_usr, name, age) VALUES (2, 'Bob', 45);

-- Table: salary
CREATE TABLE salary (
    ID_usr INT,
    year DATE,
    salary FLOAT
);
-- Sample Rows:
INSERT INTO salary (ID_usr, year, salary) VALUES (1, '2020-01-01', 50000);
INSERT INTO salary (ID_usr, year, salary) VALUES (2, '2020-01-01', 70000);

-- Table: studies
CREATE TABLE studies (
    ID INT,
    ID_usr INT,
    educational_level INT,
    institution VARCHAR(100),
    years DATE,
    speciality VARCHAR(100)
);
-- Sample Rows:
INSERT INTO studies (ID, ID_usr, educational_level, institution, years, speciality) 
VALUES (1, 1, 5, 'Harvard', '2015-01-01', 'Computer Science');
INSERT INTO studies (ID, ID_usr, educational_level, institution, years, speciality) 
VALUES (2, 2, 7, 'Stanford', '2010-01-01', 'Engineering');
""" } ]


In [4]:
context.append( {'role':'system', 'content':"""
-- Example Query 1:
-- Retrieve the name and salary of employees who earn more than 60,000.
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary > 60000;

-- Example Query 2:
-- Retrieve employees who studied at 'Harvard'.
SELECT e.name
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE st.institution = 'Harvard';

-- Example Query 3:
-- Retrieve the educational level of employees who have a salary greater than 50,000.
SELECT e.name, st.educational_level
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE s.salary > 50000;
""" } )



In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [6]:
#new
context_user = context.copy()
print(return_CCRMSQL("""YOUR QUERY HERE""", context_user))

-- Retrieve the name and age of employees who have an educational level of 7.
SELECT e.name, e.age
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE st.educational_level = 7;


In [10]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Retrieve the names and salaries of employees who earn more than 60,000", old_context_user))

This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary > 60000;
```

This SQL query retrieves the names and salaries of employees who earn more than $60,000 by joining the "employees" table with the "salary" table on the ID_usr field and filtering the results to only include salaries greater than 60,000.


In [11]:
#new
print(return_CCRMSQL("Retrieve the names and salaries of employees who earn less than 60,000", context_user))

```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary < 60000;
```


In [12]:
#old
print(return_CCRMSQL("Which institution has the highest average salary for its graduates?", old_context_user))

This is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM studies s
JOIN employees e ON s.ID_usr = e.ID_usr
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the tables `studies`, `employees`, and `salary` on the user ID to calculate the average salary for graduates of each institution. It then selects the institution with the highest average salary by ordering the results in descending order and limiting the output to the top result.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
     


In [14]:
print(return_CCRMSQL("Retrieve the names of employees who have a salary greater than 80,000.", context_user))

```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary > 80000;
```


In [15]:
print(return_CCRMSQL("List the names of employees who studied at Stanford and their salaries", context_user))

```sql
SELECT e.name, s.salary
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE st.institution = 'Stanford';
```


In [16]:
print(return_CCRMSQL("Find the average salary of employees who have a PhD degree.", context_user))

Certainly! To find the average salary of employees who have a PhD degree, you can use the following SQL query:

```sql
SELECT AVG(s.salary) AS avg_salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE st.educational_level = 7; -- Assuming PhD corresponds to educational level 7
```

This query joins the `employees`, `salary`, and `studies` tables on the appropriate keys and filters the results to only include employees with a PhD degree (assuming PhD corresponds to educational level 7). The query then calculates the average salary of these employees.


 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?